In [1]:
import qiskit
import copy
from qiskit.circuit.library import Reset, x, h, CXGate
from qiskit import QuantumCircuit
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import UnitarySynthesis, Unroll3qOrMore
from qiskit.dagcircuit import DAGCircuit, DAGOpNode, DAGOutNode
from qiskit.circuit import QuantumRegister, Qubit, ClassicalRegister, Clbit
from qiskit.circuit.library import Reset

In [31]:
# QubitReuse classes for MCMR implementation
class QubitReusePass(qiskit.transpiler.basepasses.TransformationPass):
    def __init__(self, target=None, type="default"):
        super().__init__()
        self.target = target
        self.type = type

    def run(self, dag):
        if self.type == "dual":
            result = Greedy(dag=dag, dual=True)
        elif self.type == "normal":
            result = Greedy(dag)
        else:
            regular = Greedy(dag=dag)
            dual = Greedy(dag=dag, dual=True)
            result = regular if regular.dag.num_qubits() <= dual.dag.num_qubits() else dual
        return result.dag
        
class Greedy:
    def __init__(self, dag: DAGCircuit, dual: bool = False) -> None:
        # Public variables
        self.dag = DAGCircuit()

        # Private variables
        self.__dual = dual
        self.__dag: DAGCircuit = dag.reverse_ops() if self.__dual else dag
        self.__creg = ClassicalRegister(self.__dag.num_clbits())
        self.__causal_cones: dict[int, set[Qubit]] = self.__get_causal_cones()
        self.__qubit_indices: dict[Qubit, int] = {
            qubit: i for i, qubit in enumerate(self.__dag.qubits)
        }
        self.__cl_indices: dict[Clbit, int] = {
            clbit: i for i, clbit in enumerate(self.__dag.clbits)
        }
        self.__qubit_mapping: dict[int, int] = {}
        self.__measured_qubits: list[int] = []
        self.__visited_nodes: set[DAGOpNode] = set()
        self.__current_added_qubit: int = 0

        # Initialization
        self.dag.add_creg(self.__creg)
        for index, _ in self.__causal_cones.items():
            self.__create_subpath(qubit=index)

        self.__qreg = QuantumRegister(bits=self.dag.qubits, name="q")
        self.dag.add_qreg(self.__qreg)

        if self.__dual:
            self.dag = self.dag.reverse_ops()

    def __get_causal_cones(self) -> dict[int, set[Qubit]]:
        """
        Returns a sorted dictionary with each qubit as key and
        their respective causal cone as value.
        """
        result = dict(
            sorted(
                list(
                    {
                        index: self.__dag.quantum_causal_cone(qubit)
                        for index, qubit in enumerate(self.__dag.qubits)
                    }.items()
                ),
                key=lambda item: len(item[1]),
            )
        )
        return result

    def __assign_qubit(self, index) -> None:
        """
        Check if a new qubit from the new graph needs to be
        assigned to a qubit from the old circuit.

        If so, it either picks from a measured qubit or adds a new one to the circuit.
        """
        if self.__qubit_mapping.get(index, None) is None:  # In case the qubit hasn't been assigned
            # Case measure is available
            if len(self.__measured_qubits) > 0:
                # Collect from the measured qubits queue.
                new_index = self.__measured_qubits.pop(0)
                # Applies a reset operation to set qubit.
                self.dag.apply_operation_back(
                    op=Reset(), qargs=(self.dag.qubits[new_index],), cargs=()
                )
                # Map this qubit to the new one.
                self.__qubit_mapping[index] = new_index
            else:  # Case no measured qubits available
                # Map the new qubit to the index from the current_added qubit
                self.__qubit_mapping[index] = self.__current_added_qubit
                # Increase latest added qubit index.
                self.__current_added_qubit += 1
                # Add a new qubit to the dag.
                self.dag.add_qubits([Qubit()])

    def __create_subpath(self, qubit: Qubit | int, until_node: DAGOpNode | None = None) -> None:
        """
        Recursively creates a subpath for a qubit in the circuit, based on its causal cone.
        """
        # If the provided qubit is an instance of Qubit, proceed to assign
        if isinstance(qubit, Qubit):
            self.__assign_qubit(self.__qubit_indices[qubit])
        # Else assign by index and retrieve the qubit index
        else:
            self.__assign_qubit(qubit)
            qubit = list(self.__qubit_indices)[qubit]
        # Get all nodes on wire
        queue = list(self.__dag.nodes_on_wire(qubit))
        for index, current_node in enumerate(queue):
            if current_node == until_node:  # Stop if we have reached the until node.
                break
            # If the current node has not been visited and is an instance of OpNode
            if current_node not in self.__visited_nodes and isinstance(current_node, DAGOpNode):
                # Add to the set of visited nodes.
                self.__visited_nodes.add(current_node)
                # Check if any of the qubits in qargs has not been added to the reduced circuit.
                if not current_node.op.name == "barrier":
                    for op_qubit in current_node.qargs:
                        # if self.__qubit_mapping.get(self.__qubit_indices[op_qubit], None) == None:
                        #     # If the qubit has not been added, make the recursion call,
                        #       make it stop at current node.
                        self.__create_subpath(op_qubit, until_node=current_node)
                    # Apply the operation, check for condition
                    if current_node.op.condition:
                        oper = copy.copy(current_node.op)
                        oper.condition = (self.__creg, oper.condition[1])
                    else:
                        oper = current_node.op
                    self.dag.apply_operation_back(
                        op=oper,
                        qargs=(
                            self.dag.qubits[self.__qubit_mapping[self.__qubit_indices[qbit]]]
                            for qbit in current_node.qargs
                        ),
                        cargs=(
                            self.dag.clbits[self.__cl_indices[clbit]]
                            for clbit in current_node.cargs
                        ),
                    )
                # If measuring, add the qubit to the list of available qubits.
                if (
                    not self.__dual
                    and current_node.op.name == "measure"
                    and (len(queue) > index + 1 and isinstance(queue[index + 1], DAGOutNode))
                ):
                    self.__measured_qubits.append(
                        self.__qubit_mapping[self.__qubit_indices[current_node.qargs[0]]]
                    )
            elif (
                self.__dual
                and isinstance(current_node, DAGOutNode)
                and isinstance(current_node.wire, Qubit)
            ):
                self.__measured_qubits.append(
                    self.__qubit_mapping[self.__qubit_indices[current_node.wire]]
                )

In [59]:
import qiskit
from qiskit.transpiler import basepasses
from qiskit.dagcircuit import DAGCircuit
from math import sqrt, log
import random
import copy

class Node:
    def __init__(self, circuit, parent=None, operation=None):
        self.circuit = circuit
        self.parent = parent
        self.children = []
        self.visits = 1
        self.value = 0
        self.operation = operation
        self.tree_log = {}

    def add_child(self, child):
        self.children.append(child)

    def update(self, value):
        self.value += value
        self.visits += 1

    def is_fully_expanded(self):
        return len(self.children) > 0

    def best_child(self, exploration_value=1.41):
        choices_weights = [
            c.value / c.visits + exploration_value * sqrt((2 * log(self.visits) / c.visits))
            for c in self.children
        ]
        return self.children[choices_weights.index(max(choices_weights))]

class MCTSOptimizer:
    def __init__(self,circuit: QuantumCircuit, max_depth=10):
        self.max_depth = max_depth        
        self.initial_circuit = circuit

    def run(self, dag):
        self.root = Node(dag)
        self.tree_log[self.root] = None
        self.best_solution = None
        self.best_value = float('-inf')

        for _ in range(100):
            leaf = self._select(self.root)
            self._expand(leaf)
            simulation_result = self._simulate(leaf)
            self._backpropagate(leaf, simulation_result)

        optimized_dag = self.best_solution.circuit if self.best_solution else self.root.circuit
        return optimized_dag

    def _select(self, node):
        while node.is_fully_expanded():
            node = node.best_child()
            self.tree_log[node] = node.parent
        return node

    def expand(self, node):
        # Utilize the Greedy logic for optimization
        dg = circuit_to_dag(node.circuit)
        optimized_dag = Greedy(dg) 
        optimized_dag = optimized_dag.dag
        
        # Create a new node with the optimized DAG
        new_node = Node(optimized_dag, parent=node, operation="QubitReusePass")
        node.add_child(new_node)
        self.tree_log[new_node] = node
                
    def simulate(self, circuit):        
        simulated_value = 0
        depth = 0
        while depth < self.max_depth:            
            depth += 1
            # Incremental improvement simulation
            simulated_value += depth  # Simulating improvement as depth increases
        return simulated_value

    def _backpropagate(self, node, value):
        while node is not None:
            node.update(value)
            if node.value > self.best_value:
                self.best_value = node.value
                self.best_solution = node
            node = node.parent


In [65]:
from qiskit import QuantumCircuit, ClassicalRegister
import random

# Define the number of qubits
num_qubits = 10000

# Initialize a quantum circuit with 20 qubits and a classical register with 20 bits
qc = QuantumCircuit(num_qubits, num_qubits)

# List of possible gates to apply
gates = ['h', 'x', 'y', 'z', 's', 't', 'cx']

# Apply 20 random gates to the circuit
for _ in range(10000):
    gate = random.choice(gates)
    if gate == 'cx':  # Control-X (CNOT) needs two qubits
        q1, q2 = random.sample(range(num_qubits), 2)
        qc.cx(q1, q2)
    else:  # Single qubit gates
        q = random.randint(0, num_qubits-1)
        getattr(qc, gate)(q)

# Add measurement to all qubits
for qubit in range(num_qubits):
    qc.measure(qubit, qubit)

In [66]:
from qiskit import QuantumCircuit

# Assuming the Node class and other necessary components are defined

# Initialize the MCTSOptimizer with the example quantum circuit
optimizer = MCTSOptimizer(qc)

# Method to start the optimization process (Pseudo-code)
def optimize_circuit(optimizer):
    root_node = Node(optimizer.initial_circuit)
    current_depth = 0
    
    while current_depth < optimizer.max_depth:
        optimizer.expand(root_node)
        simulated_value = optimizer.simulate(root_node)
        
        # Here you would select the best-performing node based on simulated_value
        # and make it the new root_node for the next iteration if applicable
        
        current_depth += 1
        
    # Return the optimized circuit from the best node after exiting the loop
    optimized_circuit = qiskit.converters.dag_to_circuit(root_node.best_child().circuit)
    return optimized_circuit

# Execute the optimization
optimized_circuit = optimize_circuit(optimizer)

In [74]:
len(optimized_circuit.qubits)

564